### Load the orginal dataset (XPT format)

In [79]:
import pyreadstat
df1, meta = pyreadstat.read_xport("dataset/TST_I.xpt")  # Hormone data
df2, meta = pyreadstat.read_xport("dataset/PBCD_I.xpt")  # Heavy metals
df3, meta = pyreadstat.read_xport("dataset/RHQ_I.xpt")  # Reproductive health
df4, meta = pyreadstat.read_xport("dataset/DEMO_I.xpt")  # Demographics

In [80]:
df1.to_csv("datasets-csv/TST_I.csv", index=False)
df2.to_csv("datasets-csv/PBCD_I.csv", index=False)
df3.to_csv("datasets-csv/RHQ_I.csv", index=False)
df4.to_csv("datasets-csv/DEMO_I.csv", index=False)

In [81]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [82]:
df1 = pd.read_csv("datasets-csv/TST_I.csv")
df2 = pd.read_csv("datasets-csv/PBCD_I.csv")
df3 = pd.read_csv("datasets-csv/RHQ_I.csv")
df4 = pd.read_csv("datasets-csv/DEMO_I.csv")

In [83]:
datasets = {"TST_I": df1, "PBCD_I": df2, "RHQ_I": df3, "DEMO_I": df4}
cleaned = {}

for name, df in datasets.items():
    if "SEQN" in df.columns:
        before = df.shape[0]
        df = df[~df.drop(columns=['SEQN']).isnull().all(axis=1)]
        after = df.shape[0]
        print(f"{name}: {before} → {after} rows after dropping empty ones")
        cleaned[name] = df
    else:
        print(f"⚠️ {name} has no SEQN column!")

TST_I: 8021 → 7211 rows after dropping empty ones
PBCD_I: 5815 → 5815 rows after dropping empty ones
RHQ_I: 3471 → 3120 rows after dropping empty ones
DEMO_I: 9971 → 9971 rows after dropping empty ones


In [84]:
for name, df in cleaned.items():
    df.to_csv(f"datasets-csv/cleaned/cleaned_{name}.csv", index=False)

In [85]:
cdf1 = pd.read_csv("datasets-csv/cleaned/cleaned_TST_I.csv")
cdf2 = pd.read_csv("datasets-csv/cleaned/cleaned_PBCD_I.csv")
cdf3 = pd.read_csv("datasets-csv/cleaned/cleaned_RHQ_I.csv")
cdf4 = pd.read_csv("datasets-csv/cleaned/cleaned_DEMO_I.csv")

In [86]:
# List of obvious logistics/meta columns to drop
drop_cols = [
    "SDDSRVYR",    # release cycle
    "RIDSTATR",    # interview/exam status
    "RIDEXMON",    # 6-month period
    "RIDEXAGM",    # age in months 
    
    # Interview languages 
    "SIALANG", "SIAPROXY", "SIAINTRP",
    "FIALANG", "FIAPROXY", "FIAINTRP",
    "MIALANG", "MIAPROXY", "MIAINTRP",
    "AIALANGA",
    
    # Household reference person info
    "DMDHRGND", "DMDHRAGE", "DMDHRBR4",
    "DMDHREDU", "DMDHRMAR", "DMDHSEDU",
    
    # Survey weights
    "WTINT2YR", "WTMEC2YR",
    
    # Survey design variables
    "SDMVPSU", "SDMVSTRA"
]
print("Original shape:", cdf4.shape)
# Drop them safely (only if they exist in the dataframe)
cdf4 = cdf4.drop(columns=[c for c in drop_cols if c in cdf4.columns])

print("After dropping logistics/meta cols:", cdf4.shape)


Original shape: (9971, 47)
After dropping logistics/meta cols: (9971, 23)


# Data Merging

In [87]:
# Step 1: Hormone model dataset
hormone_dataset = pd.merge(cdf2, cdf1, on="SEQN", how="inner")

# Step 2: Reproductive health model dataset
repro_dataset = pd.merge(cdf2, cdf3, on="SEQN", how="inner")

# At this point both datasets are "features + labels" ready
print("Hormone dataset shape:", hormone_dataset.shape)
print("Reproductive dataset shape:", repro_dataset.shape)

# # Step 3: add demographics to each with LEFT join
hormone_dataset = pd.merge(hormone_dataset, cdf4, on="SEQN", how="left")
repro_dataset = pd.merge(repro_dataset, cdf4, on="SEQN", how="left")

print("Hormone+Demo shape:", hormone_dataset.shape)
print("Repro+Demo shape:", repro_dataset.shape)

Hormone dataset shape: (4093, 23)
Reproductive dataset shape: (1553, 62)
Hormone+Demo shape: (4093, 45)
Repro+Demo shape: (1553, 84)


### Data merging stratergy
inner joins for guaranteeing complete datasets for heavy metals + reproductory and heavy metals + hormones datasets.

Used left joins when adding demographic data to preserve sample size.

In [88]:
hormone_dataset

,SEQN,WTSH2YR,LBXBPB,LBDBPBSI,LBDBPBLC,LBXBCD,LBDBCDSI,LBDBCDLC,LBXTHG,LBDTHGSI,...,DMDMARTL,RIDEXPRG,DMDHHSIZ,DMDFMSIZ,DMDHHSZA,DMDHHSZB,DMDHHSZE,INDHHIN2,INDFMIN2,INDFMPIR
0,83732.0,283573.311886,0.88,0.043,0.0,0.20,1.78,0.0,0.47,2.3,...,1.0,NaN,2.0,2.0,0.0,0.0,1.0,10.0,10.0,4.39
1,83733.0,50411.333995,2.60,0.126,0.0,3.53,31.41,0.0,3.08,15.4,...,3.0,NaN,1.0,1.0,0.0,0.0,0.0,4.0,4.0,1.32
2,83734.0,26293.483856,1.85,0.089,0.0,0.43,3.83,0.0,0.70,3.5,...,1.0,NaN,2.0,2.0,0.0,0.0,2.0,5.0,5.0,1.51
3,83738.0,9860.624532,0.49,0.024,0.0,0.14,1.25,0.0,0.46,2.3,...,NaN,NaN,5.0,5.0,0.0,2.0,1.0,6.0,6.0,1.18
4,83741.0,94196.553807,0.72,0.035,0.0,0.20,1.78,0.0,1.38,6.9,...,5.0,NaN,3.0,3.0,0.0,0.0,0.0,7.0,7.0,2.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4088,93693.0,14399.572323,0.59,0.028,0.0,0.12,1.07,0.0,0.33,1.6,...,NaN,NaN,4.0,4.0,1.0,2.0,0.0,5.0,5.0,0.82
4089,93694.0,13060.962444,0.69,0.033,0.0,0.16,1.42,0.0,0.70,3.5,...,NaN,NaN,7.0,7.0,0.0,4.0,0.0,3.0,3.0,0.34
4090,93699.0,9875.921047,0.37,0.018,0.0,0.07,0.62,1.0,0.48,2.4,...,NaN,NaN,5.0,5.0,0.0,2.0,1.0,15.0,15.0,4.58
4091,93700.0,77742.978398,1.07,0.052,0.0,0.67,5.96,0.0,0.82,4.1,...,1.0,NaN,5.0,5.0,0.0,3.0,0.0,1.0,1.0,0.00


In [89]:
repro_dataset

,SEQN,WTSH2YR,LBXBPB,LBDBPBSI,LBDBPBLC,LBXBCD,LBDBCDSI,LBDBCDLC,LBXTHG,LBDTHGSI,...,DMDMARTL,RIDEXPRG,DMDHHSIZ,DMDFMSIZ,DMDHHSZA,DMDHHSZB,DMDHHSZE,INDHHIN2,INDFMIN2,INDFMPIR
0,83742.0,47895.820345,0.36,0.017,0.0,0.25,2.22,0.0,1.54,7.7,...,1.0,2.0,4.0,4.0,1.0,1.0,0.0,6.0,6.0,1.03
1,83751.0,31925.673558,0.19,0.009,0.0,0.21,1.87,0.0,0.20,1.0,...,NaN,NaN,6.0,6.0,1.0,2.0,2.0,4.0,4.0,0.58
2,83757.0,23503.398195,2.25,0.109,0.0,0.18,1.60,0.0,0.56,2.8,...,4.0,NaN,5.0,5.0,1.0,0.0,0.0,5.0,5.0,0.77
3,83759.0,41519.629693,0.33,0.016,0.0,0.19,1.69,0.0,0.47,2.3,...,NaN,NaN,3.0,3.0,0.0,0.0,0.0,7.0,7.0,1.74
4,83761.0,36124.798936,1.80,0.087,0.0,1.11,9.88,0.0,3.22,16.1,...,5.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1548,93668.0,30322.313095,0.74,0.036,0.0,0.28,2.49,0.0,0.31,1.5,...,2.0,NaN,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.35
1549,93672.0,19715.131567,0.41,0.020,0.0,0.33,2.94,0.0,0.88,4.4,...,1.0,NaN,3.0,3.0,0.0,0.0,1.0,1.0,1.0,0.20
1550,93679.0,42681.553607,1.00,0.048,0.0,0.38,3.38,0.0,0.29,1.4,...,2.0,NaN,3.0,3.0,0.0,0.0,1.0,9.0,9.0,2.98
1551,93686.0,78091.970295,0.33,0.016,0.0,0.16,1.42,0.0,0.28,1.4,...,NaN,NaN,5.0,5.0,0.0,3.0,0.0,8.0,8.0,1.58


In [90]:
repro_dataset.to_csv('infertility.csv', index=False)
hormone_dataset.to_csv('hormone_levels.csv', index=False)
